# CDR Model - Doc2Vec_embeddings
* 본 모델의 소스 도메인과 타겟 도메인의 사용자 및 제품별로 문서 임베딩을 만드는 코드
* 사용자/제품별로 사용자 프로필(사용자만)/평균 평점/온라인 리뷰를 수집한 다음
<br/>
Doc2Vec 알고리즘을 적용하여 임베딩을 생성함
* 온라인 리뷰는 불용어 제거, 특수문자 제거 등의 자연어 전처리 작업을 수행함

In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 9.9 MB/s eta 0:00:00


In [ ]:
import nltk
import gensim
import contractions
import pandas as pd
import numpy as np
import os, random, re
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from sklearn.utils import shuffle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Embedding, Input, Flatten, Multiply, Dense, Activation, Concatenate, Dropout, BatchNormalization, Conv1D, GlobalAveragePooling1D, MaxPooling1D
pd.options.display.max_colwidth=20
pd.options.display.max_columns=999

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
rest = pd.read_csv('/content/drive/MyDrive/seminar/cdr_v2/yelp_rest_prepro.csv')
cafe = pd.read_csv('/content/drive/MyDrive/seminar/cdr_v2/yelp_cafe_prepro.csv')
bar = pd.read_csv('/content/drive/MyDrive/seminar/cdr_v2/yelp_bar_prepro.csv')

print(rest.shape)
print(cafe.shape)
print(bar.shape)

(1070842, 24)
(28117, 26)
(17735, 42)


In [ ]:
def friends(x):
  '''
  num_friends - 친구의 수로 표현
  '''
  friends = len(x.split(','))

  return friends

rest['num_friends'] = rest['friends'].apply(lambda x: friends(x))
bar['num_friends'] = bar['friends'].apply(lambda x: friends(x))
cafe['num_friends'] = cafe['friends'].apply(lambda x: friends(x))

In [ ]:
def add_side_info(df=None, domain='None'):
  '''
  Rating + Review + User Profile + Context
  user_id는 knowledge transfer를 위해 지금 수정X
  '''
  if domain == 'bar':
    cols = ['business_id', 'user_id', 'stars_y', 'categories', 'Alcohol', 'BestNights',
            'BestNights_Mon', 'BestNights_Tue', 'BestNights_Fri', 'BestNights_Wed',
            'BestNights_Thu','BestNights_Sun','BestNights_Sat', 'Music', 'video', 'dj',
            'background_music', 'jukebox', 'no_music', 'live', 'karaoke', 'text', 'num_friends', 'fans', 'user_votes']
    new_df = df[cols]

    for col in ['business_id', 'num_friends', 'fans', 'user_votes', 'categories', 'Alcohol', 'BestNights', 'Music']:
      new_df[col] = new_df[col].astype('category')
      new_df[col] = new_df[col].cat.codes.values

  elif domain == 'cafe':
    cols = ['business_id', 'user_id', 'stars_y', 'categories', 'OutdoorSeating', 'DriveThru',
            'text', 'num_friends', 'fans', 'user_votes']
    new_df = df[cols]

    for col in ['business_id', 'num_friends', 'fans', 'user_votes', 'categories', 'OutdoorSeating', 'DriveThru']:
      new_df[col] = new_df[col].astype('category')
      new_df[col] = new_df[col].cat.codes.values

  return new_df

rest = rest[['business_id', 'user_id', 'stars_y', 'text', 'num_friends', 'fans', 'user_votes']]
bar_con_side = add_side_info(bar, 'bar')
cafe_con_side = add_side_info(cafe, 'cafe')

<ipython-input-7-e6edf47980b1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = new_df[col].astype('category')
<ipython-input-7-e6edf47980b1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = new_df[col].cat.codes.values
<ipython-input-7-e6edf47980b1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
# Doc2Vec Embedding용 cleaned_text 생성

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(review_text):
    review_text = BeautifulSoup(review_text , "html.parser").get_text()
    review_text = contractions.fix(review_text)
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    review_text = str(review_text).lower()
    review_text = word_tokenize(review_text)
    review_text = [item for item in review_text if item not in stop_words]
    review_text = [lemmatizer.lemmatize(word = w, pos='v') for w in review_text]
    review_text = ' '.join(review_text)
    return review_text

rest['cleaned_text'] = rest['text'].apply(lambda x: clean_text(x))
cafe_con_side['cleaned_text'] = cafe_con_side['text'].apply(lambda x: clean_text(x))
bar_con_side['cleaned_text'] = bar_con_side['text'].apply(lambda x: clean_text(x))

<ipython-input-6-5d43f9d7275b>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review_text , "html.parser").get_text()
<ipython-input-6-5d43f9d7275b>:7: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review_text , "html.parser").get_text()
<ipython-input-6-5d43f9d7275b>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review_text , "html.parser").get_text()
<ipython-input-6-5d43f9d7275b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [ ]:
def tokenize_text(text):
  text = word_tokenize(text)
  text = [w for w in text if w.isalnum()]
  return text

def create_embed(df, embedding_size, window_size, min_count, epochs):
  '''
  embedding_size(=vector_size) : [8, 16, 32, 64, 128] --> default : 32
  window_size : [2,3,4,5] --> default : 5
  min_count : 2
  epochs : 30
  '''
  docs = []
  for i, row in df.iterrows():
    user_id = row['user_id']
    business_id = row['business_id']
    rating = str(row['stars_y'])
    review_text = row['cleaned_text']
    user_votes = str(row['user_votes'])
    num_friends = str(row['num_friends'])
    fans = str(row['fans'])

    user_doc = rating + ' ' + review_text + ' ' + user_votes + ' ' + num_friends + ' ' + fans
    bus_doc = rating + ' ' + review_text

    user_tokens = tokenize_text(user_doc)
    bus_tokens = tokenize_text(bus_doc)

    docs.append(TaggedDocument(user_tokens, [user_id]))
    docs.append(TaggedDocument(bus_tokens, [business_id]))

  model = Doc2Vec(docs, vector_size=embedding_size, window=window_size, min_count=min_count, workers=4, epochs=epochs)
  model.train(docs, total_examples=model.corpus_count, epochs=model.epochs)

  return model

In [ ]:
rest_doc_model = create_embed(rest, 32, 5, 2, 30)
cafe_doc_model = create_embed(cafe_con_side, 32, 5, 2, 30)
bar_doc_model = create_embed(bar_con_side, 32, 5, 2, 30)

In [ ]:
rest_doc_model.save('rest_doc_32_model')
cafe_doc_model.save('cafe_doc_32_model')
bar_doc_model.save('bar_doc_32_model')